In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/k/hafiznouman786/duplicate-question-pair/__results__.html
/kaggle/input/k/hafiznouman786/duplicate-question-pair/__notebook__.ipynb
/kaggle/input/k/hafiznouman786/duplicate-question-pair/__output__.json
/kaggle/input/k/hafiznouman786/duplicate-question-pair/custom.css
/kaggle/input/k/hafiznouman786/duplicate-question-pair/corpora/wordnet.zip
/kaggle/input/k/hafiznouman786/duplicate-question-pair/corpora/wordnet/data.adj
/kaggle/input/k/hafiznouman786/duplicate-question-pair/corpora/wordnet/LICENSE
/kaggle/input/k/hafiznouman786/duplicate-question-pair/corpora/wordnet/index.adj
/kaggle/input/k/hafiznouman786/duplicate-question-pair/corpora/wordnet/README
/kaggle/input/k/hafiznouman786/duplicate-question-pair/corpora/wordnet/verb.exc
/kaggle/input/k/hafiznouman786/duplicate-question-pair/corpora/wordnet/index.adv
/kaggle/input/k/hafiznouman786/duplicate-question-pair/corpora/wordnet/index.verb
/kaggle/input/k/hafiznouman786/duplicate-question-pair/corpora/wordnet/data.verb


In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("/kaggle/input/duplicate-question-pair/questions.csv")

In [3]:
import re
import pickle

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score

from difflib import SequenceMatcher

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Bidirectional,
    Dense, Concatenate, Dropout
)
from tensorflow.keras.models import Model


2026-02-01 19:28:08.260201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769974088.746691      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769974088.902232      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769974090.034770      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769974090.034808      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769974090.034811      24 computation_placer.cc:177] computation placer alr

In [4]:
df = df.dropna(subset=["question1", "question2"])

print(df.shape)
df.head()

(404348, 6)


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"[^a-z ]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["question1"] = df["question1"].apply(clean_text)
df["question2"] = df["question2"].apply(clean_text)


In [6]:
!pip install -q transformers datasets accelerate


In [7]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


In [8]:
class QPDataset(Dataset):
    def __init__(self, q1, q2, labels, tokenizer, max_len=64):
        self.q1 = q1
        self.q2 = q2
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.q1[idx],
            self.q2[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )

        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)

        return item


In [9]:
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-1

In [10]:
q1_train, q1_val, q2_train, q2_val, y_train, y_val = train_test_split(
    df["question1"].tolist(),
    df["question2"].tolist(),
    df["is_duplicate"].values,
    test_size=0.2,
    random_state=42
)


In [11]:
train_ds = QPDataset(q1_train, q2_train, y_train, tokenizer)
val_ds   = QPDataset(q1_val, q2_val, y_val, tokenizer)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=128)


In [12]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

epochs = 5
total_steps = epochs * len(train_loader)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()


In [13]:
patience = 3
best_val_loss = float("inf")
counter = 0

for epoch in range(epochs):

    # ---- TRAIN ----
    model.train()
    train_preds, train_labels = [], []
    train_loss_total = 0.0

    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}

        out = model(**batch)
        loss = out.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        train_loss_total += loss.item()

        preds = torch.argmax(out.logits, dim=1)
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(batch["labels"].cpu().numpy())

    train_loss = train_loss_total / len(train_loader)
    train_acc = accuracy_score(train_labels, train_preds)

    # ---- VALIDATE ----
    model.eval()
    val_preds, val_labels = [], []
    val_loss_total = 0.0

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}

            out = model(**batch)
            loss = out.loss

            val_loss_total += loss.item()

            preds = torch.argmax(out.logits, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(batch["labels"].cpu().numpy())

    val_loss = val_loss_total / len(val_loader)
    val_acc = accuracy_score(val_labels, val_preds)

    print(
        f"Epoch {epoch+1} | "
        f"Train Loss: {train_loss:.4f} | "
        f"Val Loss: {val_loss:.4f} | "
        f"Train Acc: {train_acc:.4f} | "
        f"Val Acc: {val_acc:.4f}"
    )

    # ---- EARLY STOPPING ----
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), "best_model.pt")
    else:
        counter += 1
        print(f"No improvement. Counter: {counter}/{patience}")

        if counter >= patience:
            print("Early stopping triggered 🚨")
            break


Epoch 1 | Train Loss: 0.3642 | Val Loss: 0.3067 | Train Acc: 0.8303 | Val Acc: 0.8629
Epoch 2 | Train Loss: 0.2926 | Val Loss: 0.2836 | Train Acc: 0.8703 | Val Acc: 0.8777
Epoch 3 | Train Loss: 0.2609 | Val Loss: 0.2802 | Train Acc: 0.8875 | Val Acc: 0.8808
Epoch 4 | Train Loss: 0.2415 | Val Loss: 0.2717 | Train Acc: 0.8967 | Val Acc: 0.8847
Epoch 5 | Train Loss: 0.2270 | Val Loss: 0.2726 | Train Acc: 0.9039 | Val Acc: 0.8861
No improvement. Counter: 1/3


In [14]:
print("hi")

hi
